# Swap Curve Historical Analysis & Cheap/Rich Valuation (Interactive)

## 概要
過去100日間のOISデータを用いて、以下の分析をインタラクティブに行います。

1. **データ取得 & グリッド生成**: 0.5年刻みの高解像度Forward Gridを作成
2. **PCAモデル構築**: 全期間データでPCAモデルを学習し、カーブの「構造的特徴」を抽出
3. **インタラクティブ分析**: スライダーで過去の日付を選択し、その時点での「Z-Score（統計的割高・割安）」と「PCA残差（構造的割高・割安）」をヒートマップで確認

In [ ]:
# 基本ライブラリ
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import QuantLib as ql
import ipywidgets as widgets
from ipywidgets import interact

project_root = Path("../").resolve()
sys.path.insert(0, str(project_root))

from sqlalchemy import create_engine, text
try:
    from app.core.config import settings
    DATABASE_URL = settings.database_url
    engine = create_engine(DATABASE_URL)
except Exception as e:
    print(f"DB Error: {e}")

## 1. データ取得 & グリッド生成

In [ ]:
def get_historical_irs_data(limit_days=100):
    with engine.connect() as conn:
        # SQL using single quotes, no escaping needed in JSON
        date_query = text("SELECT DISTINCT trade_date FROM irs_data WHERE product_type = 'OIS' ORDER BY trade_date DESC LIMIT :limit")
        dates_df = pd.read_sql(date_query, conn, params={'limit': limit_days})
        target_dates = tuple(dates_df['trade_date'].tolist())
        if not target_dates: return pd.DataFrame()
        dates_param = target_dates if len(target_dates) > 1 else (target_dates[0],)
        data_query = text("SELECT trade_date, tenor, rate FROM irs_data WHERE product_type = 'OIS' AND trade_date IN :dates")
        df = pd.read_sql(data_query, conn, params={'dates': dates_param})
    return df

def convert_tenor(t): 
    if 'Y' in t: return ql.Period(int(t.replace('Y', '')), ql.Years)
    if 'M' in t: return ql.Period(int(t.split('(')[0].replace('M', '')), ql.Months)
    return None

calc_starts = np.arange(0.5, 10.5, 0.5).tolist()
calc_tenors = np.arange(0.5, 10.5, 0.5).tolist() + [15.0, 20.0, 30.0]

df_hist = get_historical_irs_data(100)
history_records = []
unique_dates = sorted(df_hist['trade_date'].unique())

for d in unique_dates:
    day_data = df_hist[df_hist['trade_date'] == d]
    try:
        val_date = ql.Date(d.day, d.month, d.year)
        ql.Settings.instance().evaluationDate = val_date
        helpers = [ql.OISRateHelper(2, convert_tenor(row['tenor']), ql.QuoteHandle(ql.SimpleQuote(row['rate']/100.0)), ql.OvernightIndex("TONA", 2, ql.JPYCurrency(), ql.Japan(), ql.Actual365Fixed())) for _, row in day_data.iterrows() if convert_tenor(row['tenor'])]
        curve = ql.PiecewiseLogCubicDiscount(val_date, helpers, ql.Actual365Fixed())
        curve.enableExtrapolation()
        
        res = {'trade_date': d}
        for s in calc_starts:
            for t in calc_tenors:
                sd = val_date + ql.Period(int(s*12), ql.Months)
                res[f"{s}Y_{t}Y"] = curve.forwardRate(sd, sd + ql.Period(int(t*12), ql.Months), ql.Actual365Fixed(), ql.Compounded, ql.Annual).rate() * 100
        history_records.append(res)
    except: pass

df_fwd = pd.DataFrame(history_records).set_index('trade_date').sort_index()
print(f"Grid Ready: {df_fwd.shape}")

## 2. PCAモデル構築 & 全期間の残差計算

In [ ]:
# 1. PCA学習 (全期間)
X = df_fwd.dropna()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=3)
pca.fit(X_scaled)

# 2. 全期間の残差計算
X_reconstructed = scaler.inverse_transform(pca.inverse_transform(pca.transform(X_scaled)))
df_residuals = pd.DataFrame(X.values - X_reconstructed, index=X.index, columns=X.columns) * 100 # bp単位

# 3. 全期間のZ-Score計算
df_zscores = (df_fwd - df_fwd.mean()) / df_fwd.std()

print(f"PCA Explained Variance: {pca.explained_variance_ratio_}")
print(f"Residuals & Z-Scores pre-calculated for all {len(X)} days.")

## 3. インタラクティブ・ダッシュボード

In [ ]:
def show_dashboard(date_idx):
    target_date = df_fwd.index[date_idx]
    z_data = df_zscores.loc[target_date]
    res_data = df_residuals.loc[target_date]
    
    def to_matrix(series):
        mat = pd.DataFrame(index=calc_starts, columns=calc_tenors)
        for s in calc_starts:
            for t in calc_tenors:
                col = f"{s}Y_{t}Y"
                if col in series: mat.loc[s, t] = series[col]
        return mat.astype(float)
    
    z_mat = to_matrix(z_data)
    res_mat = to_matrix(res_data)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 10))
    sns.heatmap(z_mat, ax=ax1, cmap="RdBu_r", center=0, annot=False, cbar_kws={'label': 'Sigma'})
    ax1.set_title(f"Z-Score (Statistical Rich/Cheap)\nDate: {target_date}", fontsize=14)
    sns.heatmap(res_mat, ax=ax2, cmap="RdBu_r", center=0, annot=False, cbar_kws={'label': 'Basis Points'})
    ax2.set_title(f"PCA Residuals (Model Rich/Cheap)\nBlue=Rich, Red=Cheap", fontsize=14)
    plt.tight_layout()
    plt.show()

dates_list = df_fwd.index.tolist()
date_slider = widgets.SelectionSlider(
    options=[(d.strftime('%Y-%m-%d'), i) for i, d in enumerate(dates_list)],
    value=len(dates_list)-1,
    description='Date:',
    layout=widgets.Layout(width='80%')
)

interact(show_dashboard, date_idx=date_slider);